In [5]:
import os
import librosa
import numpy as np
import warnings
import tqdm
#import libs.functions as _fn
speaker_dir = '/mnt/nvme2tb/datasets/voxceleb2/sorted/train/id00012'
warnings.filterwarnings("ignore")
samples = os.listdir(speaker_dir)
maxs = {}
pbar = tqdm.tqdm(samples)
for f in pbar:
    audio, sr = librosa.load(os.path.join(speaker_dir,f), sr=16000, mono=True)
    s = librosa.stft(y=audio,
                     n_fft=400,
                     hop_length=160,
                     window='hamming',
                     center=True)
    S = np.abs(s)**2
    mel_basis = librosa.filters.mel(sr=sr,
                                        n_fft=400,
                                        fmin=20,
                                        fmax=8000,
                                        htk=True,
                                        n_mels=64)
    F = np.dot(mel_basis, S)
    maxs[f] = np.max(F)
    pbar.set_postfix({'current max': np.max([el for el in maxs.values()])})


100%|██████████| 159/159 [00:15<00:00, 10.25it/s, current max=172] 


In [9]:
for key, value in maxs.items():
    if value > 100:
        print(key, ":", value) 

L5fo1OvuXIg_00060.m4a : 172.01607


In [ ]:
import os
csv_dir = './diffs'
csv_list = os.listdir(csv_dir)
csvs = np.zeros((64, 0))
for f in csv_list:
    data = np.genfromtxt(os.path.join(csv_dir, f), delimiter=',', skip_header=False) # np.loadtxt(os.path.join(csv_dir, f))
    # print(data.shape)
    csvs = np.concatenate((csvs, data), axis=1)
print(csvs.shape)
np.save('combined.dat', csvs)

In [ ]:
csvs.shape
import librosa.display
librosa.display.specshow(csvs, sr=16000, hop_length=160, x_axis='time', y_axis='mel')

In [ ]:
data = csvs
gen = np.empty(data.shape)
means = np.mean(data, axis=1)
stds = np.std(data, axis=1)
for i, (mean, std) in enumerate(zip(means, stds)):
    gen[i] = np.random.normal(mean, std, data.shape[1])
librosa.display.specshow(gen, sr=16000, hop_length=160, x_axis='time', y_axis='mel')

In [ ]:
np.mean(data), np.min(data), np.max(data)


In [3]:
import librosa
import os
import random
import soundfile as sf
import string
import librosa.display
import numpy as np

signal_dir_path = '/mnt/nvme2tb/datasets/voxceleb2/sorted/train'
ambient_dir_path = '/mnt/nvme2tb/datasets/music-genres-kaggle/Data/genres_original'

signal_dirs = [f.path for f in os.scandir(signal_dir_path) if f.is_dir()]
ambient_dirs = [f.path for f in os.scandir(ambient_dir_path) if f.is_dir()]
#print(signal_dirs)

signal_index = []
for sigdir in signal_dirs:
    signal_index.extend([os.path.join(sigdir, el) for el in os.listdir(sigdir)])
#signal_index = os.listdir(signal_dir)
ambient_index = []
for ambdir in ambient_dirs:
    ambient_index.extend([os.path.join(ambdir, el) for el in os.listdir(ambdir)])

def A(raw_audio, sr=16000, n_mels=40, frame_length_ms=25, hop_ms=10):
    # 512 (32ms) vs 256 (16ms) when sr=16000 / optimal for speech is (512) 23ms with sr=22050
    # "frame length=25ms, frame shift=10ms" / 1/16000 => window = 400, hop = 160
    frame_length = int(frame_length_ms * sr / 1000)
    hop = int(hop_ms * sr / 1000)
    S = np.abs(
        librosa.stft(y=raw_audio,
                     n_fft=frame_length,
                     hop_length=hop,
                     window='hamming',
                     center=True))**2
    # Calculating mel_basis
    mel_basis = librosa.filters.mel(sr=sr,
                                    n_fft=frame_length,
                                    fmin=20,
                                    fmax=8000,
                                    htk=True,
                                    n_mels=n_mels)
    F = np.dot(mel_basis, S)
    return F


def Z(F):
    Fdb = librosa.power_to_db(F, ref=np.max)[:, :-1]
    return Fdb

def std(spg):
    return (spg + 40.) / 40.


# print(signal)  
#signal, sr = torchaudio.load(f1, normalize=True)
#transform = torchaudio.transforms.Resample(sr, 16000)
#print(signal.shape, sr)



'''
sf.write('signal.wav', signal, samplerate=sr)
sf.write('resampled.wav', signal, samplerate=sr)
#print(signal.shape, '16000')
ambient, sr = librosa.load(fa, mono=True)
librosa.resample(ambient, sr, 16000, fix=True)
ambient = ambient[:signal.shape[0]]
sf.write('ambient.wav', ambient, samplerate=sr)
'''

"\nsf.write('signal.wav', signal, samplerate=sr)\nsf.write('resampled.wav', signal, samplerate=sr)\n#print(signal.shape, '16000')\nambient, sr = librosa.load(fa, mono=True)\nlibrosa.resample(ambient, sr, 16000, fix=True)\nambient = ambient[:signal.shape[0]]\nsf.write('ambient.wav', ambient, samplerate=sr)\n"

In [4]:
#import matplotlib.pyplot as plt

for j in range(100000):
    #print(j)
    signal, sr = librosa.load(random.choice(signal_index), mono=True)
    #print(signal.shape, sr)
    librosa.resample(signal, sr, 16000, fix=True)

    ambient, sr = librosa.load(random.choice(ambient_index), mono=True)
    #print(ambient.shape, sr)
    librosa.resample(ambient, sr, 16000, fix=True)
    start = random.randint(0, ambient.shape[0]-signal.shape[0])
    ambient_cut = ambient[start:start+signal.shape[0]]

    mixed = np.add(signal*.75, ambient_cut*.25)

    #sf.write('signal.wav', signal, samplerate=22050)
    #sf.write('ambient.wav', ambient_cut, samplerate=22050)
    #sf.write('mixed.wav', mixed, samplerate=22050)

    spgd = std(Z(A(signal)))
    spgs = std(Z(A(mixed)))
    spgdiff = spgs-spgd
    letters = string.ascii_lowercase
    fname = './diffs-music-louder/' + ''.join(random.choice(letters) for i in range(10)) + '.csv'
    np.savetxt(fname, spgdiff, delimiter=",")
    #print(np.mean(spgdiff), np.var(spgdiff), np.std(spgdiff))
    #for i in range(spgdiff.shape[0]):
    #    print(np.mean(spgdiff[i]), np.var(spgdiff[i]), np.std(spgdiff[i]))
#librosa.display.specshow(spgdiff, sr=16000, hop_length=160, x_axis='time', y_axis='mel')

/home/sergey/anaconda3/envs/torch/lib/python3.9/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/home/sergey/anaconda3/envs/torch/lib/python3.9/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/home/sergey/anaconda3/envs/torch/lib/python3.9/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/home/sergey/anaconda3/envs/torch/lib/python3.9/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/home/sergey/anaconda3/envs/torch/lib/python3.9/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.


KeyboardInterrupt: 

In [ ]:
librosa.display.specshow(spgs, sr=16000, hop_length=160, x_axis='time', y_axis='mel')

In [ ]:
librosa.display.specshow(spgd, sr=16000, hop_length=160, x_axis='time', y_axis='mel')

In [3]:
import numpy as np
a = np.load('/mnt/nvme2tb/datasets/voxceleb2/spectrogram_noises/noises.npy')
b = np.load('music-louder.npy')

In [4]:
c = np.concatenate((a,b), axis=1)

In [9]:
a.shape, b.shape, c.shape
np.save('/mnt/nvme2tb/datasets/voxceleb2/spectrogram_noises/noises_louder.npy', c)